In [1]:
from langchain.retrievers import ParentDocumentRetriever
!pip install -U deeplake
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os 
os.environ["LANGCHAIN_API_KEY"] = "ls__f6cedd82954b4882a54cd5d95c278dc1"
from langchain_community.llms import Ollama 
llm = Ollama(model="mistral", temperature = 0)


In [4]:
!dir

 Volume in drive C is Windows-SSD
 Volume Serial Number is 10BB-81C2

 Directory of c:\HackFest

06-04-24  11:37    <DIR>          .
05-04-24  02:06           448,063 doc1.json
06-04-24  03:02           499,063 output.json
05-04-24  23:00            44,487 rag.ipynb
06-04-24  04:47            62,365 rag2.ipynb
05-04-24  20:05                59 requirements.txt
06-04-24  03:07    <DIR>          res
06-04-24  11:40             5,963 test.ipynb
06-04-24  01:49               696 test1.py
               7 File(s)      1,060,696 bytes
               2 Dir(s)   3,263,184,896 bytes free


In [2]:
# loaders = [
#     TextLoader("paul_graham_essay.txt"),
#     TextLoader("state_of_the_union.txt"),
# ]
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())

import os

# Get a list of all .txt files in the 'res' directory
txt_files = [f for f in os.listdir('res') if f.endswith('.txt')]

# Create a TextLoader for each file
loaders = [TextLoader(os.path.join('res', f)) for f in txt_files]

docs = []
for loader in loaders:
    docs.extend(loader.load())


In [3]:
from langchain_community.embeddings import OllamaEmbeddings
embedding = OllamaEmbeddings(model="nomic-embed-text")

In [4]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [10]:
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage import LocalFileStore


fs = LocalFileStore("./embeddings")
store = create_kv_docstore(fs)
# parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma(collection_name="split_parents", embedding_function=embedding, persist_directory="./db")
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    # parent_splitter=parent_splitter,
)
retriever.add_documents(docs, ids=None)

In [11]:
list(store.yield_keys())

['00598792-c911-4378-8a5f-2028d6c8399e',
 '00f7b839-bc55-4a92-8bde-b7c03f560ce1',
 '0102a735-6a8e-4764-a9d8-569cc7889b5b',
 '011799ad-6b33-4093-bbff-4cd836f7a4bd',
 '024ace3b-3d2d-4a26-9a48-241c86c8e048',
 '024e8b98-19e8-4f3f-a30f-972c017fc9fe',
 '029b5537-3e49-4f94-90cc-59b45485486d',
 '035ea416-9452-4fab-9ed5-6dabca9adc8c',
 '0386c229-6f50-4ba7-b327-760e7e96f8a4',
 '03978224-8885-4c79-af9c-d88f8460a82f',
 '03bb6e1e-d5ac-44a4-b755-d59991e9d642',
 '0419679b-d974-4a6b-bdc9-fb264428d90e',
 '0443d6ae-4287-40b0-aec5-f4cde4b6392d',
 '0457109e-297f-4a25-adb1-9241e18beccd',
 '04837446-fc48-4279-ae82-ddd10d925788',
 '063030ab-0b75-460c-b673-b140c6127fad',
 '0656689f-b0bf-4adf-b2d4-7e1ec79fc377',
 '06b16701-32bf-4f17-b49d-ccc6ff697bea',
 '06c6221b-97f9-421f-935a-24572278959b',
 '06e88895-f799-4e73-9f18-a58cdd949081',
 '074dace1-b045-4b97-ae05-a34d8a32cd03',
 '07d16212-a4c2-41e6-b843-703743bd7489',
 '07f63b89-694b-4d6e-9a92-669dc422480e',
 '0862ea5a-9fb9-4ecf-beaf-768d093de1bf',
 '08aca44c-2b59-

In [1]:
sub_docs = vectorstore.similarity_search("a doctor who is very bad with senior citizens")

NameError: name 'vectorstore' is not defined

In [13]:
for i in range(0,len(sub_docs)):
    print(sub_docs[i].page_content+"\n------\n")



Totally worst doctor He does not know how to talk with the patient parties and his behaviour is very very bad Ha has no patience to talk Very very bad experience He just give tests and he prefers to admit patient in his hospital on immediate basis without use medicine and proper checkup
------

Worst doctor he dont have basic sense to talk while consultation
He didnt even care to listen to whole history He just wanted to finish the appointment and move on
The most rash rude doctor who kept us waiting for more than 2 hours in spite of having an appointment
No proper response for our douts and clarification
Please dont visit this doctor
------

He is worst doctor He is money minded the doctor did not provide appropriate treatment Doesnt pay attention to his patients only intrested in number of patients and passing down patients to his son sukrit sood Never go to this doctor for treatment
------

Worst doctor he is just making the busines result are about nil or zero plz dont consult this

In [14]:
retrieved_docs = retriever.get_relevant_documents("best urologist")



In [15]:
print(retrieved_docs[0].metadata)



{'source': 'res\\579f59ec-6251-484b-9e12-34cb8a10de57.txt'}
